In [ ]:
#request 原始碼
from bs4 import BeautifulSoup as bs
import requests

url="https://www.ptt.cc/bbs/Stock/index6651.html"
headers={
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
}

res = requests.get(url , headers=headers)
print(res.text)

In [ ]:
#BeautifilSoup解析
soup=bs(res.text,"lxml")
data=soup.select("div.r-ent")
#print(data[1])
#print(len(data))
for ele in data:
    #print(ele)
    title = ele.select("div.title")[0].text
    print("標題 :",title)
    print("-"*100)

In [ ]:
#解析標題/時間/推文/連結 自訂函式

def get_parsing_data(soup):
    data = soup.select("div.r-ent")
    result = []
    
    for sample in data:
        #標題
        title = sample.select("div.title")[0].text.strip()
        
        if "公告" in title or "閒聊" in title or "創作" in title or "刪除" in title:
            continue
       
        #時間
        time = sample.select("div.date")[0].text.strip()
        
        #推文量
        push_num = sample.select("span.hl")[0].text if len(sample.select("span.hl")) > 0 else 0
        
        #作者
        author = sample.select("div.author")[0].text.strip()
        
        #連結
        raw_link = sample.select("div.title a")[0]["href"]
        domain_name = "https://www.ptt.cc"
        link = domain_name + raw_link
        result.append({
            "title": title,
            "time": time,
            "author": author,
            "push_num": push_num,
            "link": link
        })
    return result

get_parsing_data(soup)

In [ ]:
#request 原始碼
from bs4 import BeautifulSoup as bs
import requests

url="https://www.ptt.cc/bbs/Stock/index.html"
headers={
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
}

res = requests.get(url , headers = headers)
soup = bs(res.text,"lxml")
print(soup)

In [ ]:
#翻頁
previous_page = soup.select("div#action-bar-container div.btn-group-paging a")[1]["href"]
previous_page = domain_name + previous_page
print(previous_page)

page_num = (previous_page.replace("https://www.ptt.cc/bbs/Stock/index","").replace(".html",""))
for i in range(5):
    print("https://www.ptt.cc/bbs/Stock/index{}.html".format( int(page_num)-i ))

In [ ]:
#合體程式
from bs4 import BeautifulSoup as bs
import requests

#抓取首頁資料
url="https://www.ptt.cc/bbs/Stock/index.html"
headers={
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
}

domain_name = "https://www.ptt.cc"

res = requests.get(url , headers = headers)

soup = bs(res.text,"lxml")

#解析首頁資料
output = []
result = get_parsing_data(soup)
output += result

#抓取分頁資料
previous_page = soup.select("div#action-bar-container div.btn-group-paging a")[1]["href"]
previous_page = domain_name + previous_page

page_num = (previous_page.replace("https://www.ptt.cc/bbs/Stock/index","").replace(".html",""))
for i in range(5):
    url = ("https://www.ptt.cc/bbs/Stock/index{}.html".format( int(page_num)-i ))
    
    res = requests.get(url , headers = headers)
    soup = bs(res.text,"lxml")
    result = get_parsing_data(soup)
    output += result
    print("{} is ok".format(url))
print("Done")

In [20]:
#存入結果到.csv檔案中
import pandas as pd
df = pd.DataFrame(output)
df.to_csv("ptt-stock.csv", encoding="utf-8", index=False)

In [25]:
json_data = df.to_json(orient='records', force_ascii=False)  # force_ascii=False 可以保留非 ASCII 字符

# 將 JSON 保存到文件
with open('ptt-stock.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)